# Извлечение именованных сущностей.

Именованные сущности - общий термин, который используют для обозначения какого-то множества слов (словосочетаний, последовательностей символов), которые представляют какой-то особый инетерес в контексте решаемой практической задачи и которые нужно отделить от остальных слов. Это могут быть какие-то стандартные вещи: имена, фамилии, названия организаций, локации, денежные суммы, даты, номера телефонов, а могут быть и более специфичные: статьи кодексов, белки, симптомы, товары.


В многих случаях достаточно написать регулярку, чтобы решить задачу на приемлемом уровне (например собрать все номера телефонов совсем несложно). Но когда сущности выражются нестандартным образом, а множество представителей сущности постоянно расширяется - задача становится очень сложной.

Стандартные сущности ещё можно извлекать с помощью готовых инструментов.

Для английского удобно использовать [SpaCy](https://spacy.io/). Там сразу извлекаются сущности с хорошим качеством.

Для русского (если не хочется ничего делать) можно использовать тэги из pymorphy.

In [19]:
!pip install natasha ipymarkup

In [128]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [130]:
p = morph.parse('Михаил')[0].tag
print('Тэги - ', p)
print('Name' in p) #тэг имени

Тэги -  NOUN,anim,masc,Name sing,nomn
True


In [131]:
p = morph.parse('Иванов')[0].tag
print('Тэги - ', p)
print('Surn' in p) #тэг фамилии

Тэги -  NOUN,anim,masc,Sgtm,Surn sing,nomn
True


In [132]:
p = morph.parse('Петрович')[0].tag
print('Тэги - ', p)
print('Patr' in p) #тэг отчества

Тэги -  NOUN,anim,masc,Patr sing,nomn
True


In [137]:
p = morph.parse('Одинцово')[0].tag
print('Тэги - ', p)
print('Geox' in p) #тэг локация

Тэги -  NOUN,inan,neut,Sgtm,Geox sing,nomn
True


In [138]:
p = morph.parse('Яндекс')[0].tag
print('Тэги - ', p)
print('Orgn' in p) #тэг организация

Тэги -  NOUN,inan,masc,Orgn sing,nomn
True


In [142]:
p = morph.parse('МГУ')[0].tag
print('Тэги - ', p)
print('Orgn' in p) #тэг организация

Тэги -  NOUN,inan,masc,Sgtm,Fixd,Abbr,Orgn sing,gent
True


In [129]:
p = morph.parse('ВШЭ')[0].tag
print('Тэги - ', p)
print('Orgn' in p) #тэг организация

Тэги -  UNKN
False


## &uarr;&uarr;&uarr;
### Попробуйте другие слова для этих типов сущностей, чтобы понять, что работает, а что нет

Рядом стоящие слова одного тэга можно склеить в один. Или сначала собрать нграмы и если какое-то одно слово в нграмме принадлежит к какому-то типу сущности, то распространить его на весь нграм.

Другой инструмент - natasha (https://github.com/natasha/natasha)

До недавнего времени это была просто библиотека с правилами, написанными на yargy https://github.com/natasha/yargy (специальной грамматике на питоне), а теперь это уже многоцелевая русскоязычная нлп-библиотека. Извлекать сущности тут можно либо предобученной моделью либо правилами.

In [130]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    Doc,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,
    
)


# для красивой отрисовки есть отдельная библиотека
from ipymarkup import show_span_box_markup


# интерфейс загрузки моделей пока не самый простой
# со временем наверное станет получше
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)


# правила загружаются вот так
names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

In [131]:
text = 'Влад Веселов. Андрей Петрович. Алиса Викторовна. Студия Артемия Лебедева'
# для моделей нужно превратить текст в класс Doc (похоже на spacy)
doc = Doc(text)


# потом модели применяются к тексту
doc.segment(segmenter) # сегментер нужно применить, иначе нер не будет работать
doc.tag_ner(ner_tagger)
show_span_box_markup(text, doc.spans)

In [132]:
matches = names_extractor(text)
spans = [(_.start, _.stop, 'PER') for _ in matches]
show_span_box_markup(text, spans)

In [147]:
text = 'Более того в Москве, на улице Такой. На реке Оке. В германии'

doc = Doc(text)
doc.segment(segmenter)
doc.tag_ner(ner_tagger)
show_span_box_markup(text, doc.spans)

In [148]:
matches = addr_extractor(text)
spans = [(_.start, _.stop, 'LOC') for _ in matches]
show_span_box_markup(text, spans)

In [81]:
text = 'ФСБ. Московский государственный университет. Высшая школа экономики. ВШЭ. Mail.ru '
doc = Doc(text)
doc.segment(segmenter)
doc.tag_ner(ner_tagger)
show_span_box_markup(text, doc.spans)

In [138]:
text = 'С 2015 г. по 2017 год. 16 апреля 1993 года. В четверг . 23.12.18'

matches = dates_extractor(text)
spans = [(_.start, _.stop) for _ in matches]
show_span_box_markup(text, spans)

In [146]:
text = "Он заплатил ему 3,000,000,000 рублей."

matches = money_extractor(text)
spans = [(_.start, _.stop) for _ in matches]
show_span_box_markup(text, spans)

In [143]:
text = "Он заплатил ему 3000 000 000 рублей."

matches = money_extractor(text)
spans = [(_.start, _.stop) for _ in matches]
show_span_box_markup(text, spans)

Ещё есть томита-парсер, но с ним очень тяжело работать (никакого развития, скудная документация, закрытый код, никакого сообщества) https://tech.yandex.ru/tomita/

Если нужно улучшить выделение стандартныйх типов или научиться извлекать специфичные сущности, нужно писать правила в Yargy. 

Возьмем тексты объявлений на авито по категории Консоли и попробуем научиться выделять названия приставок Xbox и Playstation.

In [46]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

/Users/mnefedov/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


Посмотрим на данные.

In [48]:
data = pd.read_csv('pristavki.csv', header=None, names=['text'])

In [49]:
data.shape

(9285, 1)

Давайте попробуем сначала полные варианты: Xbox 360, Xbox one, Playstation 1,2,3,4.

In [50]:
from yargy import Parser, rule, or_
from yargy.predicates import in_, in_caseless
from yargy.tokenizer import MorphTokenizer
from yargy.pipelines import morph_pipeline, caseless_pipeline
from yargy.interpretation import fact
from IPython.display import display

Для начала определим саму сущность, которую будет извлекать. Сущность будет называться Pristavka и неё будет два атрибута: название и версия. Делается это вот так:

In [51]:

Pristavka = fact(
    'Pristavka',
    ['name', 'model']
)

(Чуть позже станет понятнее зачем так делать)

Теперь сделаем правило для иксбокса. Просто напишем несколько вариантов написания Xbox и версий.
Такой список называется Газзетир. В Yargy его удобно задавать через morph_pipeline. Туда можно написать слова или последовательности, а он их нормализует, чтобы потом сопоставлять с текстом.


В итоге у нас два газзетира. Мы оборачиваем их в правило (rule). Работать это будет примерно как регулярка. 

In [165]:
Xbox = rule(
    morph_pipeline(['Xbox', 'X box', "Иксбокс"])
    .interpretation(Pristavka.name), # то, что сматчиться будет в атрибуте name
    morph_pipeline(['360', 'one'])
    .interpretation(Pristavka.model)) # то, что сматчиться будет в атрибуте version


Правило для плейстешена будет аналогичное.

In [166]:
PS = rule(
    morph_pipeline(['Playstation', 'Play station', 'PS']).interpretation(Pristavka.name),
    morph_pipeline(['1', '2', '3', '4']).interpretation(Pristavka.model)    
    )

Теперь сделаем общее правило, которое будет искать плейстешены или иксбоксы и извлекать их как сущность типа Pristavka.

In [167]:
PRISTAVKA = or_(PS, Xbox).interpretation(Pristavka)

parser = Parser(PRISTAVKA) # создаем парсер, которым будем проходить по тексту

Попробуем сматчить по текстам.

In [168]:
matches = []

for sent in data.text[:100]:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [169]:
for m in matches[:10]:
    print(m.name, m.model)

PS 3
Ps 4
PS 1
PS 3
PlayStation 3
PS 3
Xbox 360
Playstation 3
Ps 4
ps 2


У приставок ещё могут быть подификации вроде Slim, X, S и т.д, но не всегда. Чтобы учесть это можно добавить ещё одно правило с .optional() на конце.

In [170]:
Pristavka = fact(
    'Pristavka',
    ['name', 'model','version']
)

Xbox = rule(
    morph_pipeline(['Xbox', 'X box', "Иксбокс"]).interpretation(Pristavka.name),
    morph_pipeline(['360', 'one']).interpretation(Pristavka.model),
    morph_pipeline(['s', 'x', 'e']).interpretation(Pristavka.version).optional()) 

PS = rule(
    morph_pipeline(['Playstation', 'Play station', 'PS']).interpretation(Pristavka.name),
    morph_pipeline(['1', '2', '3', '4']).interpretation(Pristavka.model),
    morph_pipeline(['Slim', 'SuperSlim', 'слим']).interpretation(Pristavka.version).optional()
    )
PRISTAVKA = or_(PS, Xbox).interpretation(Pristavka)

parser = Parser(PRISTAVKA) # создаем парсер, которым будем проходить по тексту

In [171]:
list(parser.findall('Playstation 3 slim'))[0].fact

Pristavka(
    name='Playstation',
    model='3',
    version='slim'
)

In [172]:
list(parser.findall('Playstation 3 арарв'))[0].fact

Pristavka(
    name='Playstation',
    model='3',
    version=None
)

In [92]:
matches = []

for sent in data.text[:1000]:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [96]:
len(matches)

381

In [102]:
for m in matches[:10]:
    print(m.name, m.model)

PS 3
Ps 4
PS 1
PS 3
PlayStation 3
PS 3
Xbox 360
Playstation 3
Ps 4
ps 2


Все работает как и должно.

Хорошо бы ещё приводить все варианты названия к нормальному виду. Вот как это можно сделать.

Если можно заменить весь газзетир на какое-то одно слово, то можно просто добавить в интерпретации значение .сonst('something')

In [121]:
Pristavka = fact(
    'Pristavka',
    ['name', 'model','version']
)

Xbox = rule(
    morph_pipeline(['Xbox', 'X box', "Иксбокс"]).interpretation(Pristavka.name.const('Xbox')),
    morph_pipeline(['360', 'one']).interpretation(Pristavka.model),
    morph_pipeline(['s', 'x', 'e']).interpretation(Pristavka.version).optional()) 

PS = rule(
    morph_pipeline(['Playstation', 'Play station', 'PS']).interpretation(Pristavka.name.const('Playstation')),
    morph_pipeline(['1', '2', '3', '4']).interpretation(Pristavka.model),
    morph_pipeline(['Slim', 'SuperSlim', 'слим']).interpretation(Pristavka.version).optional()
    )
PRISTAVKA = or_(PS, Xbox).interpretation(Pristavka)

parser = Parser(PRISTAVKA) # создаем парсер, которым будем проходить по тексту

In [122]:
matches = []

for sent in data.text[:200]:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [123]:
for m in matches[:10]:
    print(m.name, m.model)

Playstation 3
Playstation 4
Playstation 1
Playstation 3
Playstation 3
Playstation 3
Xbox 360
Playstation 3
Playstation 4
Playstation 2


В общем случае к нормальной форме приводит .normalized() добавленный к атрибуту в интерпретации. Но он работает через pymorphy2, а он работает с русским языком. Поэтому в нашем случае его можно использовать только для привода к нижнему регистру.

Когда у каждого и слов в газзетире есть своя нормальная форма, можно сделать газзетир словарём, где ключи это нужные формы, а значения - нормальные формы. А в интерпретации к .normalized() добавить .custom() и через него дергать нужную правильную форму.

In [112]:
Pristavka = fact(
    'Pristavka',
    ['name', 'model','version']
)

VERSIONS = {
    'super slim': 'SuperSlim',
    'superslim': 'SuperSlim',
    'slim': 'Slim',
    'fat': 'Fat',
    'pro': 'PRO',
    'vita': 'VITA'
}

Xbox = rule(
    morph_pipeline(['Xbox', 'X box', "Иксбокс"]).interpretation(Pristavka.name.const('Xbox')),
    morph_pipeline(['360', 'one']).interpretation(Pristavka.model.normalized()),
    in_caseless('sxe').interpretation(Pristavka.version.normalized()).optional()) 

PS = rule(
    morph_pipeline(['Playstation', 'Play station', 'PS']).interpretation(Pristavka.name.const('Playstation')),
    # для простоты можно написать вот так
    in_('1234').interpretation(Pristavka.model.normalized()),
    #изменения вот тут                                                      вот тут дергаем правильную форму
    caseless_pipeline(VERSIONS).interpretation(Pristavka.version.normalized().custom(VERSIONS.get)).optional()
    )
PRISTAVKA = or_(PS, Xbox).interpretation(Pristavka)

parser = Parser(PRISTAVKA) # создаем парсер, которым будем проходить по тексту

In [188]:
list(parser.findall('PS 3 SlIm  PS 3 SlIm PS 3 SlIm'))[1].fact

Pristavka(
    name='PS',
    model='3',
    version='SlIm'
)

In [174]:
list(parser.findall('XbOx 360 X'))[0].fact

Pristavka(
    name='XbOx',
    model='360',
    version='X'
)

In [118]:
matches = []

for sent in data.text:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [119]:
for m in matches[:10]:
    print(m.name, m.model)

Playstation 3
Playstation 4
Playstation 1
Playstation 3
Playstation 3
Playstation 3
Xbox 360
Playstation 3
Playstation 4
Playstation 2


In [120]:
len(matches)

3351

Улучшения для этого типа сущностей довольно очевидные и простые - нужно расширять газзетиры, можно добавить другие приставки вроде денди и нинтендо.

Для сущностей, которые нельзя перечислить словарём, нужны контекстные правила. В целом они не сильно сложнее того, что мы сейчас написали. Возможно вам понадобятся другие полезные штуки из Yargy (предикат - and_, учёт грамматической информации - gram('NOUN'), согласование по роду, числу и падежу - gnc).

Учить сразу все не имеет смысла. Возьмите практическую задачу, документацию к Yargy (https://yargy.readthedocs.io/ru/latest/index.html) и разбирайте только то, что вам нужно. 